In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import cv2
from PIL import Image

In [3]:
%conda install openpyxl

Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
 - anaconda
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/konovalyuk/anaconda3/envs/cancerPOC

  added / updated specs:
    - openpyxl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       h7b6447c_0          78 KB
    ca-certificates-2023.12.12 |       h06a4308_0         126 KB
    certifi-2024.2.2           |  py310h06a4308_0         159 KB
    debugpy-1.6.7              |  py310h6a678d5_0         2.0 MB
    et_xmlfile-1.1.0           |  py310h06a4308_0          10 KB
    importlib-metadata-6.0.0   |  py310h06a4308_0          39 KB
    ld_impl_linux-64-2.38      |       h1181459_1         654 KB
    libstdcxx-ng-11.2.0        |       h1234567_1         4.7 MB
    libuuid-1.41.5             |       h5eee18b_0   

In [5]:
data = pd.read_excel('../PH2Dataset/PH2_dataset.xlsx')
data.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,NaN,NaN,NaN,NaN,NaN,Legends,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Asymmetry,0,Fully Simmetric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,1,Symmetric in 1 axes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,2,Fully Asymmetric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Pigment Network\nDots/Globules\nStreaks\nRegre...,AT,Atypical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,T,Typical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,A,Absent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,P,Present,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data = data[12:].reset_index(drop=True)
data.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,IMD003,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,NaN,X,NaN,NaN
1,IMD009,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,NaN,NaN,NaN
2,IMD016,NaN,X,NaN,NaN,0,T,T,A,A,A,NaN,NaN,X,X,NaN,NaN
3,IMD022,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,NaN,NaN,NaN
4,IMD024,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,X,NaN,NaN
5,IMD025,NaN,X,NaN,NaN,0,T,T,A,A,A,NaN,NaN,X,NaN,NaN,NaN
6,IMD035,NaN,X,NaN,NaN,2,T,A,A,A,A,NaN,X,X,NaN,NaN,NaN
7,IMD038,NaN,X,NaN,NaN,0,T,T,A,A,A,NaN,NaN,NaN,X,NaN,X
8,IMD042,NaN,X,NaN,NaN,0,T,T,A,A,A,NaN,NaN,X,X,NaN,NaN
9,IMD044,NaN,X,NaN,NaN,0,T,T,A,A,A,NaN,NaN,NaN,X,X,NaN


In [18]:
data = data.rename(columns={'Unnamed: 0': 'image_name', 'Unnamed: 1': 'diagnosis', 'Unnamed: 2': 'nevus', 'Unnamed: 3':'atyp_nevus', 
                            'Unnamed: 4': 'melanoma', 'Unnamed: 5': 'asymmetry', 'Unnamed: 6': 'pigment', 'Unnamed: 7': 'dots',
                            'Unnamed: 8': 'streaks', 'Unnamed: 9': 'regression_areas', 'Unnamed: 10': 'blue_whitish_veil', 
                            'Unnamed: 11': 'white', 'Unnamed: 12': 'red', 'Unnamed: 13': 'light_brown', 'Unnamed: 14': 'dark_brown',
                            'Unnamed: 15': 'blue_gray', 'Unnamed: 16': 'black'})

In [37]:
data

,image_name,diagnosis,nevus,atyp_nevus,melanoma,asymmetry,pigment,dots,streaks,regression_areas,blue_whitish_veil,white,red,light_brown,dark_brown,blue_gray,black
0,IMD003,NaN,1,0,0,0,T,A,A,A,A,0,0,0,1,0,0
1,IMD009,NaN,1,0,0,0,T,A,A,A,A,0,0,1,0,0,0
2,IMD016,NaN,1,0,0,0,T,T,A,A,A,0,0,1,1,0,0
3,IMD022,NaN,1,0,0,0,T,A,A,A,A,0,0,1,0,0,0
4,IMD024,NaN,1,0,0,0,T,A,A,A,A,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,IMD424,Melanoma,0,0,1,2,AT,AT,P,A,P,0,0,1,1,1,1
196,IMD425,Melanoma,0,0,1,2,AT,AT,A,P,P,0,0,1,1,1,1
197,IMD426,Melanoma,0,0,1,2,AT,A,A,P,A,0,0,1,1,0,0
198,IMD429,NaN,0,0,1,0,AT,A,P,A,P,0,0,1,1,1,1


In [50]:
data.black[data.nevus == 'X'] = 1
data.black[data.melanoma == 0] = 0

/tmp/ipykernel_398244/2396287588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bin['nevus'].loc[data_bin.nevus == 1] = 1.
/tmp/ipykernel_398244/2396287588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bin['nevus'].loc[data_bin.nevus == 0] = 0.


In [40]:
data.melanoma.value_counts(), data.nevus.value_counts()

(melanoma
 0    160
 1     40
 Name: count, dtype: int64,
 nevus
 0    120
 1     80
 Name: count, dtype: int64)

In [42]:
data.to_csv('../PH2Dataset/PH2_dataset.csv', index=False)

In [70]:
data_bin = data[['image_name', 'nevus', 'melanoma']]
data_bin.head(10)

,image_name,nevus,melanoma
0,IMD003,1,0
1,IMD009,1,0
2,IMD016,1,0
3,IMD022,1,0
4,IMD024,1,0
5,IMD025,1,0
6,IMD035,1,0
7,IMD038,1,0
8,IMD042,1,0
9,IMD044,1,0


In [79]:
data_bin.nevus = data_bin.nevus.astype(float)
data_bin.melanoma = data_bin.melanoma.astype(float)

In [80]:
data_bin

,image_name,nevus,melanoma
0,IMD003,1.0,0.0
1,IMD009,1.0,0.0
2,IMD016,1.0,0.0
3,IMD022,1.0,0.0
4,IMD024,1.0,0.0
...,...,...,...
195,IMD424,0.0,1.0
196,IMD425,0.0,1.0
197,IMD426,0.0,1.0
198,IMD429,0.0,1.0


In [82]:
data_bin.to_csv('../PH2Dataset/binary_labels.csv', index=False)